In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import statsmodels.api as sm
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio
import statsmodels.api as sm

from sklearn.preprocessing import PolynomialFeatures
from plotly.subplots import make_subplots
pio.renderers.default = 'iframe'
# Set notebook mode to work in offline
pyo.init_notebook_mode()

data = pd.read_csv("/home/jessica/reading-kiln-termostat/data/analises_curvas.csv")
data['index'] = data.index

/tmp/ipykernel_4289/3649226079.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def create_curve_array(data_alpha):
    curves = []
    T_spacing = [1160, 1180, 1200]
    for index, row in data_alpha.iterrows():
        row = row.astype(float)
        curve_array = []
        curve_array = np.append(curve_array, segment_array(T_spacing[0], T_spacing[1], row.alpha_3))
        curve_array = np.append(curve_array, segment_array(T_spacing[1], T_spacing[2], row.alpha_4))
        curve_array = np.append(curve_array, segment_array(T_spacing[2], row['temp final'], row.alpha_5))
        curve_array = np.append(curve_array, [row['temp final']] * row.patamar.astype(int))
        curves.append(curve_array)
    return curves

def segment_array(initial_temp, final_temp, alpha):
    r = np.floor(np.arange(initial_temp, np.floor(final_temp), alpha))
    if r[-1] == np.floor(final_temp):
         return r
    else:
        return np.append(r, np.floor(final_temp))
    
cone_dict = {
    "rampa" : [150, 60, 40, 30, 15],
    "temp_final": [1240, 1220, 1210, 1205, 1190]
}
tab_data = pd.DataFrame(cone_dict)
tab_data['alpha'] = tab_data['rampa'] / 60
tab_data['patamar'] = 0

curves_std = []
for index, row in tab_data.iterrows():
    row = row.astype(float)
    curve = []
    curve = np.append(curve, segment_array(1160, row['temp_final'] + 0.5, row.alpha))
    curves_std.append(curve)
    
tab_data['sinal'] = curves_std

energia = []
for curve in curves_std:
    energia = np.append(energia,np.sum(curve))
    
tab_data['energia_SUM'] =  energia

In [4]:
exp_data = data.filter(regex='alpha|index|temp final|patamar|cone_meio')
exp_data['index'] = exp_data.reset_index().index
exp_data = exp_data.dropna()
for column in exp_data.columns:
    exp_data = exp_data.drop(exp_data[exp_data[column].astype(str).str.contains('-')].index)
    
    
exp_data = exp_data.astype(float)

/tmp/ipykernel_4289/917373620.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
curves_exp = create_curve_array(exp_data)
energia = []

for curve in curves_exp:
    energia = np.append(energia,np.sum(curve))
    
exp_data['energia_SUM'] =  energia
exp_data['sinal'] = curves_exp

In [6]:
exp_data

,temp final,patamar,alpha_100,cone_meio,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,index,energia_SUM,sinal
1,1220.0,20.0,1.20,7.5,1.18,1.30,1.20,1.05,0.83,1.0,101903.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1166...."
2,1220.0,20.0,1.20,7.4,1.54,1.30,1.07,0.87,0.77,2.0,109002.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165...."
3,1236.0,0.0,2.50,7.6,1.67,1.25,1.03,0.83,0.71,3.0,118822.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165...."
5,1220.0,10.0,1.20,7.2,1.33,1.30,1.07,1.00,1.20,5.0,82362.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165...."
6,1220.0,10.0,1.20,6.6,1.54,1.36,1.07,0.95,0.77,6.0,95622.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165...."
7,1220.0,10.0,1.20,7.2,1.54,1.36,1.00,0.77,0.57,7.0,113648.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165...."
12,1212.0,3.0,1.12,7.0,1.33,1.20,0.94,0.80,0.52,12.0,91611.0,"[1160.0, 1160.0, 1161.0, 1162.0, 1163.0, 1164...."
14,1212.0,3.0,1.12,7.2,1.25,1.20,0.81,0.67,0.57,14.0,98659.0,"[1160.0, 1160.0, 1161.0, 1162.0, 1163.0, 1164...."


In [7]:
def best_model(signal):
    return np.sum([((s - 1175) ** 2) + 0.3 * s  for s in signal])

In [8]:
exp_data['best_energy'] = exp_data['sinal'].apply(best_model)
exp_data

,temp final,patamar,alpha_100,cone_meio,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,index,energia_SUM,sinal,best_energy
1,1220.0,20.0,1.20,7.5,1.18,1.30,1.20,1.05,0.83,1.0,101903.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1166....",110314.9
2,1220.0,20.0,1.20,7.4,1.54,1.30,1.07,0.87,0.77,2.0,109002.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....",113675.6
3,1236.0,0.0,2.50,7.6,1.67,1.25,1.03,0.83,0.71,3.0,118822.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....",144015.6
5,1220.0,10.0,1.20,7.2,1.33,1.30,1.07,1.00,1.20,5.0,82362.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....",74283.6
6,1220.0,10.0,1.20,6.6,1.54,1.36,1.07,0.95,0.77,6.0,95622.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....",89542.6
7,1220.0,10.0,1.20,7.2,1.54,1.36,1.00,0.77,0.57,7.0,113648.0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....",108319.4
12,1212.0,3.0,1.12,7.0,1.33,1.20,0.94,0.80,0.52,12.0,91611.0,"[1160.0, 1160.0, 1161.0, 1162.0, 1163.0, 1164....",63475.3
14,1212.0,3.0,1.12,7.2,1.25,1.20,0.81,0.67,0.57,14.0,98659.0,"[1160.0, 1160.0, 1161.0, 1162.0, 1163.0, 1164....",65243.7


In [9]:
tab_data['best_energy'] = tab_data['sinal'].apply(best_model)
tab_data

,rampa,temp_final,alpha,patamar,sinal,energia_SUM,best_energy
0,150,1240,2.500000,0,"[1160.0, 1162.0, 1165.0, 1167.0, 1170.0, 1172....",39592.0,50806.6
1,60,1220,1.000000,0,"[1160.0, 1161.0, 1162.0, 1163.0, 1164.0, 1165....",72590.0,54412.0
2,40,1210,0.666667,0,"[1160.0, 1160.0, 1161.0, 1162.0, 1162.0, 1163....",90035.0,50385.5
3,30,1205,0.500000,0,"[1160.0, 1160.0, 1161.0, 1161.0, 1162.0, 1162....",107585.0,52765.5
4,15,1190,0.250000,0,"[1160.0, 1160.0, 1160.0, 1160.0, 1161.0, 1161....",142130.0,51884.0
